In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
!pip install -q transformers[sentencepiece] ohmeow-blurr fastai nbdev

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.1/66.1 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.5/887.5 MB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 35.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.7/58.7 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 9.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 48.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
!pip install -q onnx
!pip install -q onnxruntime
!pip install -q onnxruntime-gpu
!pip install -q onnxruntime_tools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 48.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 16.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.1/157.1 MB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.7/212.7 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 3.7 MB/s eta 0:00:00


In [ ]:
import torch
import numpy as np
import pandas as pd
import onnxruntime as rt
from sklearn import metrics
from fastai.text.all import *
from tqdm.notebook import tqdm
from blurr.text.data.all import *
from blurr.text.modeling.all import *
from transformers import AutoTokenizer
from onnxruntime.quantization import QuantType
from onnxruntime.quantization import quantize_dynamic

/usr/local/lib/python3.10/dist-packages/blurr/text/modeling/question_answering.py:31: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  squad_metric = load_metric("squad")
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for squad contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/squad/squad.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [ ]:
from google.colab import drive
drive.mount('/content/drive', timeout_ms = 30000000)

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/keyword_categorization/

/content/drive/.shortcut-targets-by-id/18aTrjN2Ugh0bCahBW3L1xSwAl-MNEXVD/keyword_generation


In [ ]:
df = pd.read_csv("data/papers_final_data.csv")

In [ ]:
for index in range(len(df)):
  df.iloc[index]['final_keywords'] = [ item.replace(" '","").replace("'","").lower() for item in re.split(",", df.iloc[index]['final_keywords'].replace("[","").replace("]","")) ]

In [ ]:
available_keywords = {}

keywords_list = df['final_keywords'].to_list()

for item in range(len(keywords_list)):
  for keyword in keywords_list[item]:
    if keyword in available_keywords.keys():
      available_keywords[keyword] += 1
    else:
      available_keywords[keyword] = 1

In [ ]:
encoding_path = "encoding"

In [ ]:
encode_keywords = { key: idx for idx, (key, value) in enumerate(available_keywords.items())}
with open(f"{encoding_path}/encoded_keywords.json", "w") as fp:
  json.dump(encode_keywords, fp)

In [ ]:
categorical_keywords_list = []

for keywords in keywords_list:
  categorical_list = [0]*len(encode_keywords)
  for keyword in keywords:
    keyword_index = encode_keywords[keyword]
    categorical_list[keyword_index] = 1
  categorical_keywords_list.append(categorical_list)

In [ ]:
df['encoded_keywords'] = categorical_keywords_list

In [ ]:
splitter = RandomSplitter(valid_pct = 0.1, seed = 42)
train_ids, valid_ids = splitter(df)

In [ ]:
valid_df = df.loc[valid_ids]

In [ ]:
def metric_measures(test_df, preds, model_name):

  targets = [np.asarray(target) for target in test_df['encoded_keywords'].to_list()]

  # print(f"Targets: {targets[0]}, Datatype: {type(targets[0])}")

  outputs = [np.asarray(pred) for pred in preds]

  # print(f"Outputs: {outputs[0]}, Datatype: {type(outputs[0])}")

  accuracy = metrics.accuracy_score(targets, outputs)

  f1_score_micro = metrics.f1_score(targets, outputs, average = 'micro')

  f1_score_macro = metrics.f1_score(targets, outputs, average = 'macro')

  print(f"{model_name}'s evaluation metrics:")

  print(f"Clasification matrix: {metrics.classification_report(targets, outputs, digits=6)}")

  print(f"\tOverall Accuracy: {accuracy*100}%")

  # Micro F1-score is more accurate for multi-label classification
  print(f"\tF1 Score (Micro) = {f1_score_micro*100}%")

  # Macro F1-score is more accurate for single-label classification
  print(f"\tF1 Score (Macro) = {f1_score_macro*100}%")

  return

In [ ]:
model_dir = "models"

model_name = "bert.pkl"

learner = load_learner(f"{model_dir}/{model_name}")

In [ ]:
onnx_model = "bert.onnx"

In [ ]:
torch_model = learner.model.hf_model.eval()

torch.onnx.export(
    torch_model,
    torch.LongTensor([[0]*512]),
    f"{model_dir}/{onnx_model}",
    input_names = ['input_ids'],
    output_names = ['output'],
    opset_version = 17,
    dynamic_axes = {
        'input_ids': {
            0: 'batch_size',
            1: 'sequence_len',
        },
        'output': {
            0: 'batch_size'
        }
    }
)

In [ ]:
quantized_model = "bert_quantized.onnx"

In [ ]:
quantize_dynamic(
    f"{model_dir}/{onnx_model}",
    f"{model_dir}/{quantized_model}",
    weight_type = QuantType.QUInt8
)

Ignore MatMul due to non constant B: /[/bert/encoder/layer.0/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.0/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.1/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.1/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.2/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.2/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.3/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.3/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.4/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.4/attention/self/MatMul_1]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.5/attention/self/MatMul]
Ignore MatMul due to non constant B: /[/bert/encoder/layer.5/atten

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

class_labels = list(encode_keywords.keys())

inf_session = rt.InferenceSession(f'{model_dir}/{quantized_model}')
input_name = inf_session.get_inputs()[0].name
output_name = inf_session.get_outputs()[0].name

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
preds = []
for idx, row in tqdm(valid_df.iterrows()):
  # print(row)
  desc = row['abstracts']
  # print(f"\n{desc}")

  input_ids = tokenizer(desc)['input_ids'][:512]

  # print(f"\n{input_ids}")

  probs = inf_session.run([output_name], {
      input_name: [input_ids]
  })[0]

  probs = torch.FloatTensor(probs)

  # print(f"{probs}")

  masks = torch.sigmoid(probs) >= 0.5

  # print(f"{masks}")

  # labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]

  # print(labels)

  # pred_genres = [0] * len(encode_keywords)

  # print(np.asarray(masks.long()[0]))

  preds.append(np.asarray(masks.long()[0]))



0it [00:00, ?it/s]

In [ ]:
# preds = []
# for idx, row in tqdm(valid_df.iterrows(), total=10):

#   desc = row['abstracts']

#   input_ids = tokenizer(desc)['input_ids'][:512]

#   print(f"Input Id's length: {len(input_ids)}")

#   probs = inf_session.run([output_name], {
#       input_name: [input_ids]
#   })[0]

#   probs = torch.FloatTensor(probs)

#   masks = torch.sigmoid(probs) >= 0.5

#   labels = [class_labels[idx] for idx, mask in enumerate(masks[0]) if mask]


#   # labels = model.blurr_predict(desc)[0]['class_indices']
#   pred_genres = [0] * len(encode_keywords)
#   for label in labels:
#     pred_genres[encode_keywords[label]] = 1
#   preds.append(labels)

# preds[0][:20]

  0%|          | 0/10 [00:00<?, ?it/s]

Input Id's length: 161
Input Id's length: 320
Input Id's length: 122
Input Id's length: 367
Input Id's length: 216
Input Id's length: 342
Input Id's length: 91
Input Id's length: 307
Input Id's length: 192
Input Id's length: 251
Input Id's length: 194
Input Id's length: 189
Input Id's length: 271
Input Id's length: 145
Input Id's length: 287
Input Id's length: 228
Input Id's length: 37
Input Id's length: 239
Input Id's length: 179
Input Id's length: 234
Input Id's length: 138
Input Id's length: 146
Input Id's length: 228
Input Id's length: 201
Input Id's length: 292
Input Id's length: 54
Input Id's length: 249
Input Id's length: 174
Input Id's length: 302
Input Id's length: 108
Input Id's length: 248
Input Id's length: 314
Input Id's length: 287
Input Id's length: 103
Input Id's length: 304
Input Id's length: 75
Input Id's length: 231


KeyboardInterrupt: 

In [ ]:
metric_measures(valid_df, preds, model_name="BERT")

deBERTa's evaluation metrics:
Clasification matrix:               precision    recall  f1-score   support

           0   0.746269  0.568182  0.645161        88
           1   0.766667  0.718750  0.741935        32
           2   0.891892  0.868421  0.880000        38
           3   0.695652  0.317881  0.436364       151
           4   0.428571  0.348837  0.384615        43
           5   0.333333  0.024390  0.045455        41
           6   0.481928  0.476190  0.479042        84
           7   0.725000  0.491525  0.585859        59
           8   0.796875  0.662338  0.723404        77
           9   0.644444  0.487395  0.555024       119
          10   0.698630  0.621951  0.658065        82
          11   0.608696  0.424242  0.500000        33
          12   0.561404  0.410256  0.474074        78
          13   0.564103  0.478261  0.517647        46
          14   0.711864  0.608696  0.656250       138
          15   0.771429  0.177632  0.288770       152
          16   0.476190  0.12

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
